# MedImageInsight for Image and Text Embeddings Deployment and Inference using Online Endpoints

This sample shows how to deploy MedImageInsight embedding type models to an online endpoint for image and text embeddings inference. For this notebook, we use Python 3.10 - SDK v2. 

### Task
MedImageInsight takes in images and/or text samples. For each image and text sample, feature embeddings are returned from the model.
 
### Model
The models that can perform the `embeddings` task are tagged with `embeddings`. We will use the `MedImageInsight` model in this notebook. 

### Inference data
We will use a chest X-ray image and text as a sample input. 

### Outline
1. Setup pre-requisites
2. Pick a model to deploy
3. Deploy the model to an online endpoint
4. Test the endpoint
5. Clean up resources - delete the endpoint

## 1. Setup pre-requisites
* Install [Azure ML Client library for Python](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-ml-readme?view=azure-python)
* Connect to AzureML Workspace and authenticate.

In [7]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
)

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

## 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `embeddings` task. In this example, we use the `MedImageInsight` model. If you have opened this notebook for a different model, replace the model name accordingly.

In [10]:
# The models are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(
    credential,
    registry_name="azureml",
)
model = registry_ml_client.models.get(name="MedImageInsight", label="latest")
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json


## 3. Deploy the model to an online endpoint for real time inference
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [11]:
import random
import string

endpoint_name = "medimageinsight"

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = f"{endpoint_name}-{endpoint_suffix}"

print(f"Endpoint name: {endpoint_name}")

Endpoint name: medimageinsight-u2g5q


In [15]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [16]:
from azure.ai.ml.entities import OnlineRequestSettings

deployment_name = "medimageinsight-v1"
deployment_package = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model,
    instance_type="STANDARD_NC4AS_T4_V3",  # Use a GPU instance type like STANDARD_NC4AS_T4_V3 for fast inference
    instance_count=1,
    request_settings=OnlineRequestSettings(request_timeout_ms=90000),
    app_insights_enabled=True,
)

In [17]:
ml_client.online_deployments.begin_create_or_update(deployment_package).result()

Check: endpoint medimageinsight-u2g5q exists


.........................................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'medimageinsight-u2g5q', 'type': 'Managed', 'name': 'medimageinsight-v1', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6c180dd2-1ec4-4fad-8ba8-1f2d8d67c129/providers/Microsoft.MachineLearningServices/locations/westus2/mfeOperationsStatus/odidp:681e8849-345b-4da8-b1b6-8697ba1ef038:d510a9dd-8938-4983-9e3d-e7abe2addacb?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/6c180dd2-1ec4-4fad-8ba8-1f2d8d67c129/resourceGroups/fmmg-mars-collab/providers/Microsoft.MachineLearningServices/workspaces/fmmg-mars-collab/onlineEndpoints/medimageinsight-u2g5q/deployments/medimageinsight-v1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/jmerkow-cpu4/code/Users/jmerkow/healthcareai-azureml-examples/sdk/python/foundation-model

## 4 Test the endpoint - base64 encoded image and text

We will fetch a sample chest X-ray image and submit to online endpoint for inference.

<img src="https://openi.nlm.nih.gov/imgs/512/145/145/CXR145_IM-0290-1001.png" alt="CXR Image" width="200">

In [18]:
import urllib.request

image_url = "https://openi.nlm.nih.gov/imgs/512/145/145/CXR145_IM-0290-1001.png"
image_path = "CXR145_IM-0290-1001.png"

urllib.request.urlretrieve(image_url, image_path)
print(f"Image downloaded to {image_path}")

Image downloaded to CXR145_IM-0290-1001.png


In [19]:
import base64
import json
import os

sample_image_xray = os.path.join(image_path)


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()


data = {
    "input_data": {
        "columns": ["image", "text"],
        #  IMPORTANT: Modify the index as needed
        "index": [0],
        "data": [
            [
                base64.encodebytes(read_image(sample_image_xray)).decode("utf-8"),
                "x-ray chest anteroposterior Pneumonia",
            ]
        ],
    },
    "params": {"get_scaling_factor": True},
}

# Create request json
request_file_name = "sample_request_data.json"
with open(request_file_name, "w") as request_file:
    json.dump(data, request_file)

In [20]:
# Score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file=request_file_name,
)
print(f"raw response: {response}\n")

raw response: [{"image_features": [[0.029661938548088074, -0.027228673920035362, -0.009731546975672245, 0.017045041546225548, -0.018564002588391304, 0.04184061288833618, -0.010718000121414661, 0.005454281345009804, 0.021867165341973305, -0.017360005527734756, -0.04865431785583496, -0.01143177505582571, -0.037878844887018204, -0.04433659091591835, -0.021754402667284012, 0.049756456166505814, -0.0062940688803792, -0.013364968821406364, 0.03862381353974342, 0.03412209078669548, 0.0247862096875906, -0.011875680647790432, 0.012414560653269291, 0.015322347171604633, -0.00592561112716794, 0.007902690209448338, -0.044613324105739594, 0.01912156119942665, 0.02469729818403721, -0.026587314903736115, 0.02233879081904888, 0.002249476034194231, -0.05268988758325577, 0.006737730465829372, -0.00267909187823534, 0.03224223107099533, -0.01761448010802269, -0.01492451410740614, 0.0059272246435284615, -0.0375819094479084, 0.05240040272474289, 0.008582129143178463, -0.043645989149808884, 0.007506014313548

In [21]:
# Convert result to numpy array
result = json.loads(response)
print("Features type", result[0].keys())

for feature in result[0].keys():
    print(f"Feature: {feature}")
    print(result[0][feature])

Features type dict_keys(['image_features', 'text_features'])
Feature: image_features
[[0.029661938548088074, -0.027228673920035362, -0.009731546975672245, 0.017045041546225548, -0.018564002588391304, 0.04184061288833618, -0.010718000121414661, 0.005454281345009804, 0.021867165341973305, -0.017360005527734756, -0.04865431785583496, -0.01143177505582571, -0.037878844887018204, -0.04433659091591835, -0.021754402667284012, 0.049756456166505814, -0.0062940688803792, -0.013364968821406364, 0.03862381353974342, 0.03412209078669548, 0.0247862096875906, -0.011875680647790432, 0.012414560653269291, 0.015322347171604633, -0.00592561112716794, 0.007902690209448338, -0.044613324105739594, 0.01912156119942665, 0.02469729818403721, -0.026587314903736115, 0.02233879081904888, 0.002249476034194231, -0.05268988758325577, 0.006737730465829372, -0.00267909187823534, 0.03224223107099533, -0.01761448010802269, -0.01492451410740614, 0.0059272246435284615, -0.0375819094479084, 0.05240040272474289, 0.008582129

## 5. Clean up resources - delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name).wait()